In [1]:
import os
import json
import pandas as pd
from components import db_functions

In [11]:

db_file = os.path.join('data','db','proteogyver.db')
db_conn = db_functions.create_connection(db_file)
new_current = db_functions.get_full_table_as_pd(db_conn, 'common_proteins')
db_conn.close()

In [9]:
to_modify = []
for _,row in current[current['protein_type'].str.contains('ipynb')].iterrows():
    to_modify.append([row['uniprot_id'], row['protein_type'].replace('.ipynb_checkpoints, ', '')])
db_conn = db_functions.create_connection(db_file)
for mod in to_modify:
    db_functions.modify_record(db_conn, 'common_proteins', 'uniprot_id', mod[0], ['protein_type'], [mod[1]])
db_conn.commit()
db_conn.close()

In [15]:
new_current['protein_type'].value_counts()

protein_type
Proteasome                                           474934
Keratin                                               75348
Elongation factor                                      5290
Ribosome                                               3942
Carboxylase                                            2816
Chaperone                                              2351
Histone                                                2017
Heat shock protein                                     1373
Actin                                                  1207
Chaperone, Heat shock protein                          1135
Tubulin                                                 671
Heat shock protein, Proteasome                          117
Chaperone, Histone                                       89
Keratin, Proteasome                                      71
Elongation factor, Ribosome                              40
Chaperone, Proteasome                                    40
Chaperone, Tubulin         

In [14]:
current['protein_type'].value_counts().sum()

571622

In [4]:
current.head()

,uniprot_id,gene_name,entry_name,all_gene_names,organism,protein_type,update_time
0,P04792,HSPB1,HSPB1_HUMAN,HSPB1 HSP27 HSP28,Homo sapiens (Human),"Heat shock protein, Proteasome",2024-11-21
1,P63284,clpB,CLPB_ECOLI,clpB htpM b2592 JW2573,Escherichia coli (strain K12),"Chaperone, Heat shock protein",2024-11-21
2,P10591,SSA1,HSP71_YEAST,SSA1 YAL005C,Saccharomyces cerevisiae (strain ATCC 204508 /...,"Heat shock protein, Proteasome",2024-11-21
3,Q00613,HSF1,HSF1_HUMAN,HSF1 HSTF1,Homo sapiens (Human),"Heat shock protein, Proteasome",2024-11-21
4,O14558,HSPB6,HSPB6_HUMAN,HSPB6,Homo sapiens (Human),Heat shock protein,2024-11-21


In [6]:
current[current['gene_name'].fillna('').str.contains('ATP')]

,uniprot_id,gene_name,entry_name,all_gene_names,organism,protein_type,update_time
1372,P27449,ATP6V0C,VATL_HUMAN,ATP6V0C ATP6C ATP6L ATPL,Homo sapiens (Human),Proteasome components,2024-02-16
3407,Q9NQ11,ATP13A2,AT132_HUMAN,ATP13A2 PARK9,Homo sapiens (Human),Proteasome components,2024-02-16
3966,P23956,ATP6V0C,VATL_BOVIN,ATP6V0C ATP6C ATP6L,Bos taurus (Bovine),Proteasome components,2024-02-16
5297,O18882,ATP6V0C,VATL_SHEEP,ATP6V0C ATP6C ATP6L,Ovis aries (Sheep),Proteasome components,2024-02-16
7747,Q9LU63,ATP1,ATP1_ARATH,ATP1 PDL1 SDIRIP1 At5g51110 MWD22.5,Arabidopsis thaliana (Mouse-ear cress),Carboxylases,2024-02-16


In [ ]:
comdir = os.path.join('data_assets','db build files','Potential contaminant protein groups')
comdir = '/home/kmsaloka/Downloads/Database additions/Potential contaminant protein groups'
db_file = os.path.join('data','proteogyver.db')

com_cols = [
    'uniprot_id TEXT PRIMARY KEY',
    'gene_name TEXT',
    'entry_name TEXT',
    'all_gene_names TEXT',
    'organism TEXT',
    'protein_type TEXT NOT NULL',
    'update_time TEXT NOT NULL '
]
db_conn = db_functions.create_connection(db_file)
current = db_functions.get_full_table_as_pd(db_conn, 'common_proteins')
have = set(current['uniprot_id'].values)
db_conn.close()
common_proteins = {}
for root, dirs, files in os.walk(comdir):
    for f in files:
        comdf = pd.read_csv(os.path.join(root,f),sep='\t')
        name = root.rsplit(os.sep,maxsplit=1)[-1]
        for _,row in comdf.iterrows():
            if row['Entry'] not in common_proteins:
                common_proteins[row['Entry']] = [
                    row['Entry'],
                    row['Gene Names (primary)'],
                    row['Entry Name'],
                    row['Gene Names'],
                    row['Organism'],
                    [name],
                    datetime.today().strftime('%Y-%m-%d')
                ]
            else:
                common_proteins[row['Entry']][5].append(name)

In [ ]:
modifications = []
additions = []
for common_protein, data in common_proteins.items():
    if common_protein in have:
        cur_ver = current[current['uniprot_id']==common_protein]
        mods = cur_ver['protein_type'].split(', ')
        mods.extend(data[5])
        modifications.append([common_protein, 'protein_type', ', '.join(sorted(list(set(mods))))])
    else:
        data[5] = ', '.join(data[5])
        additions.append([com_cols, data])
db_conn = db_functions.create_connection(db_file)
for addcols, add_data in additions:
    db_functions.add_record(db_conn, 'common_proteins', addcols, add_data)

In [12]:
current.head()

,uniprot_id,gene_name,entry_name,all_gene_names,organism,protein,update_time
0,A0A0G2JZ79,Sirt1,SIR1_RAT,Sirt1,Rattus norvegicus (Rat),Keratin,2024-02-05
1,A0A0H2URK1,psrP,PSRP_STRPN,psrP SP_1772,Streptococcus pneumoniae serotype 4 (strain AT...,Keratin,2024-02-05
2,D3ZER2,Bfsp2,BFSP2_RAT,Bfsp2,Rattus norvegicus (Rat),Keratin,2024-02-05
3,F7EQ49,GPER1,GPER1_MACMU,GPER1 GPER GPR30,Macaca mulatta (Rhesus macaque),Keratin,2024-02-05
4,O08878,Gper1,GPER1_RAT,Gper1 Cmkrl2 Gper Gpr30 Gpr41,Rattus norvegicus (Rat),Keratin,2024-02-05


In [21]:
def rename_column(db_conn, tablename, old_col, new_col):
    sql_str = f'ALTER TABLE {tablename} RENAME COLUMN {old_col} TO {new_col};'
    try:
        db_conn.cursor().execute(sql_str)
    except sqlite3.Error as error:
        print(f'Failed to rename column in sqlite table. Error: {error}', sql_str)
        raise
db_file = os.path.join('data','proteogyvertesting.db')
con = db_functions.create_connection(db_file)
rename_column(con, 'common_proteins','protein','protein_type')
con.commit()
con.close()

In [20]:
import sqlite3
con = sqlite3.connect(db_file)
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
con.close()

[('control_sets',), ('crapome_sets',), ('proteins',), ('control_vl_gfp_mac3_10min_ap_overall',), ('control_vl_gfp_mac3_10min_ap',), ('control_vl_gfp_mac3_10min_bioid_overall',), ('control_vl_gfp_mac3_10min_bioid',), ('control_vl_gfp_mac2_18h_ap_overall',), ('control_vl_gfp_mac2_18h_ap',), ('control_vl_gfp_mac2_18h_bioid_overall',), ('control_vl_gfp_mac2_18h_bioid',), ('control_vl_gfp_mac_24h_ap_overall',), ('control_vl_gfp_mac_24h_ap',), ('control_vl_gfp_mac_24h_ap_nls_overall',), ('control_vl_gfp_mac_24h_ap_nls',), ('control_vl_gfp_mac_24h_bioid_overall',), ('control_vl_gfp_mac_24h_bioid',), ('control_vl_gfp_mac_24h_bioid_nls_overall',), ('control_vl_gfp_mac_24h_bioid_nls',), ('crapome_vl_gfp_mac3_10min_ap',), ('crapome_vl_gfp_mac3_10min_bioid',), ('crapome_vl_gfp_mac2_18h_ap',), ('crapome_vl_gfp_mac2_18h_bioid',), ('crapome_vl_gfp_mac_24h_ap',), ('crapome_vl_gfp_mac_24h_ap_nls',), ('crapome_vl_gfp_mac_24h_bioid',), ('crapome_vl_gfp_mac_24h_bioid_nls',), ('crapome_nesvilab',), ('conta